# LLM 모델 다운로드 및 추론

In [ ]:
!pip install --upgrade torch transformers accelerate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 99.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 108.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.40.0
    Uninstalling transformers-4.40.0:
      Successfully uninstalled transformers-4.40.0


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM

In [ ]:
model_id = 'Bllossom/llama-3.2-Korean-Bllossom-3B'

In [ ]:
torch.cuda.is_available()

True

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.7k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.bfloat16,
    device_map="auto",
)

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/180 [00:00<?, ?B/s]

In [ ]:
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

messages = [
    {"role": "user", "content": f"{instruction}"}
    ]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


철수가 20개의 연필을 가지고 있었고 영희가 절반을 가져가면, 영희가 가져간 연필의 갯수는 20/2 = 10개입니다.

철수가 남은 연필의 갯수는 20 - 10 = 10개입니다. 

민수가 남은 5개의 연필을 가져가면, 철수가 남은 연필의 갯수는 10 - 5 = 5개입니다.

따라서 철수가 남은 연필의 갯수는 5개입니다.


In [ ]:
tokenizer.decode(input_ids[0])

"<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nCutting Knowledge Date: December 2023\nToday Date: 17 Nov 2024\n\nYou are a helpful AI assistant. Please answer the user's questions kindly. 당신은 유능한 AI 어시스턴트 입니다. 사용자의 질문에 대해 친절하게 답변해주세요.<|eot_id|><|start_header_id|>user<|end_header_id|>\n\n철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n"

In [ ]:
instruction = "철수가 20개의 연필을 가지고 있었는데 영희가 절반을 가져가고 민수가 남은 5개를 가져갔으면 철수에게 남은 연필의 갯수는 몇개인가요?"

# Instruction을 직접 encode
input_ids = tokenizer.encode(
    instruction,
    return_tensors="pt"
).to(model.device)

# 종료 토큰 정의
terminators = [
    tokenizer.convert_tokens_to_ids("<|end_of_text|>"),
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

# 모델 생성
outputs = model.generate(
    input_ids,
    max_new_tokens=1024,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9
)

# 출력 디코딩
print(tokenizer.decode(outputs[0][input_ids.shape[-1]:], skip_special_tokens=True))


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


 철수가 20개의 연필을 가지고 있었고 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가 10개를 가져갔을 때 철수가 남은 연필의 갯수는 10개입니다. 영희가

In [ ]:
def check_gpu_memory():
    if torch.cuda.is_available():
        gpu_count = torch.cuda.device_count()
        for i in range(gpu_count):
            total_memory = torch.cuda.get_device_properties(i).total_memory / 1e9  # GB 단위로 변환
            reserved_memory = torch.cuda.memory_reserved(i) / 1e9  # GB 단위
            allocated_memory = torch.cuda.memory_allocated(i) / 1e9  # GB 단위
            free_memory = reserved_memory - allocated_memory  # 실제 사용 가능한 메모리
            print(f"GPU {i}:")
            print(f"  Total Memory: {total_memory:.2f} GB")
            print(f"  Reserved Memory: {reserved_memory:.2f} GB")
            print(f"  Allocated Memory: {allocated_memory:.2f} GB")
            print(f"  Free Memory: {free_memory:.2f} GB\n")
    else:
        print("CUDA is not available on this system.")

In [ ]:
check_gpu_memory()

GPU 0:
  Total Memory: 15.84 GB
  Reserved Memory: 6.77 GB
  Allocated Memory: 6.43 GB
  Free Memory: 0.33 GB



# 프롬프트 작성 요령

### 구체적으로 지시해야함.

생성 모델이므로 지시한 어휘와 내용을 참조하는데, 파라미터에 들어간 지식보다 훨씬 강력하게 영향을 준다.


### 모호한 지시는 피하는것이 좋음.

 "설명은 간결하게 몇 문장으로만 하고 너무 자세히는 하지 말아 줘."
 와 같은 지시는 사실상 의미가 없을 수 있다.


 ### 하지 말아야할 것에 대한 지시는 피하는것이 좋음.

 Transformer 기반 Decoder 모델이므로, 하지 말아야할것에 대한 지시문에 대해 부정어 보다 하지 말아야 한다고 설명한 키워드에 Attention score가 더 높거나 비슷하게 가게 될 경우 오히려 하게될 가능성이 크다.